<a href="https://colab.research.google.com/github/WeatherlyMed/AutomatedGramStainInterpertation/blob/main/Automated.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# For tips on running notebooks in Google Colab, see
# https://pytorch.org/tutorials/beginner/colab
%matplotlib inline

In [ ]:
import torch
import torchvision.transforms as transforms
import os
from torchvision import datasets
from torch.utils.data import DataLoader
from google.colab import drive
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


In [ ]:


# Mount Google Drive
drive.mount('/content/drive')

# Define the root directory of your dataset
root_dir = '/content/drive/MyDrive/Sorted/'

# Define transformations
transform = transforms.Compose([
    transforms.Resize((720, 576)),  # Adjust size as needed
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])



Mounted at /content/drive


In [ ]:
batch_size = 10

trainset = datasets.ImageFolder(root=root_dir, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = datasets.ImageFolder(root=root_dir, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=True, num_workers=2)

classes = ('Negative', 'None')

In [ ]:
import os
print(os.listdir(root_dir))

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc_input_size = 399312
        self.fc1 = nn.Linear(self.fc_input_size, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 2)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

device = torch.device("cuda" if torch.cuda.is_available() else "cpu" )
net = Net().to(device)

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.09)

In [ ]:
for i in range(4):
  for epoch in range(15):  # loop over the dataset multiple times

      running_loss = 0.0
      var = enumerate(trainloader, 0)
      for i, data in var:
          # get the inputs; data is a list of [inputs, labels]
          inputs, labels = data
          inputs = inputs.to(device)
          labels = labels.to(device)
          # zero the parameter gradients
          optimizer.zero_grad()

          # forward + backward + optimize
          outputs = net(inputs)
          loss = criterion(outputs, labels)
          loss.backward()
          optimizer.step()

          # print statistics
          running_loss += loss.item()
          if i % 100 == 99:    # print every 100 mini-batches
              print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 100:.3f}')
              running_loss = 0.0
              PATH = './cifar_nets'+str(i)+'.pth'
              torch.save(net.state_dict(), PATH)

  print('Finished Training')
  PATH = './cifar_net.pth'
  torch.save(net.state_dict(), PATH)
  print('Saved')

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


[1,   100] loss: 0.683
[1,   200] loss: 0.679
[1,   300] loss: 0.676
[1,   400] loss: 0.672
[1,   500] loss: 0.669
[1,   600] loss: 0.676
[1,   700] loss: 0.674
[1,   800] loss: 0.672


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


[2,   100] loss: 0.666
[2,   200] loss: 0.669
[2,   300] loss: 0.673
[2,   400] loss: 0.683
[2,   500] loss: 0.684
[2,   600] loss: 0.671
[2,   700] loss: 0.667
[2,   800] loss: 0.675
[3,   100] loss: 0.677


In [ ]:
net = Net()
net.load_state_dict(torch.load('./cifar_net.pth'))

In [ ]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in testloader:
        images, labels = data
        # calculate outputs by running images through the network
        outputs = net(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')

In [ ]:
# prepare to count predictions for each class
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

# again no gradients needed
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predictions = torch.max(outputs, 1)
        # collect the correct predictions for each class
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1


# print accuracy for each class
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')